<a href="https://colab.research.google.com/github/liuzheqi0723/capstone-fraud-detection/blob/YaoW/models/3_FurtherProcess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Application for real-time fraudulent transaction detection**


##PART III: Further Process of the Datasets

###1. Import Dataset and Libraries

Dataset: <br>
  [link to Kaggle](https://www.kaggle.com/c/ieee-fraud-detection/data)<br>
  [link to Google Drive](https://drive.google.com/drive/folders/1_h09YxVU0BH4KgUy9su49zo3XX59oZ3d)

In [1]:
### import libraries ###

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# # # Run it if it is the first time you running this notebook.

# # # Mount your google drive to colab
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# Before you run the code below,
# Please create a shortcut for the 'Capstone' folder from shared drive to your own Googledrive.


clean_id = pd.read_csv('/content/drive/MyDrive/Capstone/Data/clean_train_id.csv')
clean_id.name = 'clean_id'
# clean_train_id.head()

clean_trans = pd.read_csv('/content/drive/MyDrive/Capstone/Data/clean_train_trans.csv')
clean_trans.name = 'clean_trans'
# clean_trans.head()

# Dataset is now stored in a Pandas Dataframe

###2. Import functions defined previously

In [4]:
# define functions used in the func 'df_description' in the next cell.
from pandas.core.frame import DataFrame

def null_info(df, axis=0):
  '''
  Return a DataFrame describes the Nans in df.
  df: DataFrame
  '''

  df_null = df.isnull().sum(axis=axis).to_frame()
  df_null.rename(columns={0: '#_Nans'}, inplace=True)
  if axis==0:
    num_total = df.shape[0]
  else:
    num_total = df.shape[1]

  df_null['%_Nans'] = round(df_null['#_Nans']/num_total*100, 2)
  df_null['#_Nans'] = df_null['#_Nans'].astype('int')

  return df_null



def dtype_info(df):
  '''
  Return a DataFrame describes the data type in df.
  df: DataFrame
  '''

  df_type = df.dtypes.to_frame()
  df_type.rename(columns={0: 'data_type'}, inplace=True)
 
  return df_type



def cols_info_df(df):
  '''
  Return a DataFrame describes the cols in df.
  df: DataFrame
  '''
  # generate decribing dfs using funcs defined above
  df_null = null_info(df) 
  df_type = dtype_info(df)

  # merge the decription dfs
  cols_info_df = df_null.merge(df_type, left_index=True, right_index=True,)  
 
  return cols_info_df


In [5]:
### Define functions used to clean dataset ###

from pandas.core.frame import DataFrame

def clean_df(df: DataFrame, \
             cols_drop: list, rows_drop: list, cols_to_category: list):
  '''
  Return a cleaned df.
  '''

  cleaned_df = df
  cleaned_df = cleaned_df.drop(columns=cols_drop) # drop cols
  cleaned_df = cleaned_df.drop(labels=rows_drop, axis=0) # drop rows

  for col in cols_to_category: # convert dtypes
    if col in cleaned_df.columns:
      cleaned_df[col] = cleaned_df[col].astype('category')

  return cleaned_df



def get_dummies(df: DataFrame, cols: list):
  '''
  Get the dummy values for the categorical columns.
  Append the it to the input df and drop the original cols.

  df: data.
  cols: the name of the columns need to be converted.
  '''

  for col in cols:
    if col in df.columns:
      col_dummies = pd.get_dummies(data=df[col])
      df = pd.concat([df, col_dummies], axis=1)
      df = df.drop(col, axis=1)

  
  return df

###3. Fill null values_A_with string and constant values. 

**'clean_id' dataset:**
1. All the Nans in columns with data type of **'object'** will be filled with string **'NA'**.
2. All the Nans in columns with data type of **'float'** will be filled with mean value of the column using [sklearn.impute](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer).<br>To prevent from data leakage, we will do this after seperating the training and testing datasets.

In [6]:
# # draft code, see if the method works
# clean_id_obj = clean_id.select_dtypes(include = ['object'])
# clean_id_obj.columns
# clean_id_obj = clean_id_obj.fillna('NA')
# clean_id_obj['id_16'].unique()

In [7]:
clean_id.drop(columns=['Unnamed: 0'], inplace=True) # drop index col

In [8]:
# fill object cols with str 'NA'
cols = clean_id.columns.to_list()
for col in cols:
  if clean_id[col].dtype == 'O':
    clean_id[col].fillna('NA', inplace=True)

In [9]:
clean_id

,TransactionID,id_01,id_02,id_05,id_06,id_11,id_12,id_13,id_15,id_16,...,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,100.0,NotFound,NaN,New,NotFound,...,144.0,New,NotFound,samsung browser 6.2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,0.0,-5.0,100.0,NotFound,49.0,New,NotFound,...,500.0,New,NotFound,mobile safari 11.0,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,100.0,NotFound,52.0,Found,Found,...,142.0,Found,Found,chrome 62.0,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,0.0,-6.0,100.0,NotFound,52.0,New,NotFound,...,507.0,New,NotFound,chrome 62.0,F,F,T,T,desktop,NA
4,2987016,0.0,7460.0,1.0,0.0,100.0,NotFound,NaN,Found,Found,...,575.0,Found,Found,chrome 62.0,T,F,T,T,desktop,MacOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134828,3577521,-15.0,145955.0,0.0,0.0,100.0,NotFound,27.0,Found,Found,...,139.0,Found,Found,chrome 66.0 for android,F,F,T,F,mobile,F3111 Build/33.3.A.1.97
134829,3577526,-5.0,172059.0,1.0,-5.0,100.0,NotFound,27.0,New,NotFound,...,333.0,New,NotFound,chrome 55.0 for android,T,F,T,F,mobile,A574BL Build/NMF26F
134830,3577529,-20.0,632381.0,-1.0,-36.0,100.0,NotFound,27.0,New,NotFound,...,411.0,New,NotFound,chrome 65.0 for android,F,F,T,F,mobile,Moto E (4) Plus Build/NMA26.42-152
134831,3577531,-5.0,55528.0,0.0,-7.0,100.0,NotFound,27.0,Found,Found,...,368.0,Found,Found,chrome 66.0,T,F,T,F,desktop,MacOS


In [10]:
# # output
# clean_id.to_csv('clean_id.csv')
# !cp clean_id.csv "drive/MyDrive/Capstone/Data/"

**'clean_trans' dataset:**



In [11]:
# # get a list with the names of the cols which contain Nan values.
# clean_trans_col = cols_info_df(clean_trans)
# col_nan = clean_trans_col[clean_trans_col.loc[:,'#_Nans']> 0].index.to_list()
# # col_nan

In [12]:
clean_trans.head()

,Unnamed: 0,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
clean_trans.drop(columns=['Unnamed: 0'], inplace=True) # drop index col

1. All the Nans in columns with data type of **'object'** will be filled with string **'NA'**.

2. For the Nans in columns with data type of **'float'** will be treated differently.<br><br>
`    2.a fill with an unique value that has never appears in the column.`

  >Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.
For example, how many times the payment card associated with a IP and email or address appeared in 24 hours time range, etc.

*Because the 'VXXX' columns are engineered features, the nan values indicate that the row do not belongs any category of the column, which is also an infomative message. *

In [14]:
# fill object cols with str 'NA'
cols = clean_trans.columns.to_list()
for col in cols: 
  if clean_trans[col].dtype == 'O': # condition 1
    clean_trans[col].fillna('NA', inplace=True)
  elif str(col).startswith( 'V' ): # condition 2.a
    clean_trans[col].fillna(-1, inplace=True)

In [15]:
clean_trans['card4'].unique()

array(['discover', 'mastercard', 'visa', 'american express', 'NA'],
      dtype=object)

    2.b fill with **mean** value of the column using sklearn.impute.

>card1 - card6: payment card information.Such as card type, card category, issue bank, country, etc.

>addr: both addresses are for purchaser.
addr1 as billing region.
addr2 as billing country.

>dist: distances between (not limited) billing address, mailing address, zip code, IP address, phone area, etc.

*To prevent from data leakage, we will do this after seperating the training and testing datasets.*<br>

    2.c fill with **most frequent value** in the column.
  >C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
  
  >D1-D15: timedelta, such as days between previous transaction, etc.

  *To prevent from data leakage, we will do this after seperating the training and testing datasets.*<br>

In [16]:
# # output
# clean_trans.to_csv('clean_trans.csv')
# !cp clean_trans.csv "drive/MyDrive/Capstone/Data/"

###4. Join datasets, Define X and y, Get dummies

In [17]:
# Join two dfs, to get a df ready for used in scikit learn.
df_join = clean_id.merge(clean_trans, left_on='TransactionID', right_on='TransactionID')
df_join
# df_join.columns.to_list() # used to check the names of each column.

,TransactionID,id_01,id_02,id_05,id_06,id_11,id_12,id_13,id_15,id_16,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987004,0.0,70787.0,NaN,NaN,100.0,NotFound,NaN,New,NotFound,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2987008,-5.0,98945.0,0.0,-5.0,100.0,NotFound,49.0,New,NotFound,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2987010,-5.0,191631.0,0.0,0.0,100.0,NotFound,52.0,Found,Found,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,2987011,-5.0,221832.0,0.0,-6.0,100.0,NotFound,52.0,New,NotFound,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,2987016,0.0,7460.0,1.0,0.0,100.0,NotFound,NaN,Found,Found,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134828,3577521,-15.0,145955.0,0.0,0.0,100.0,NotFound,27.0,Found,Found,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
134829,3577526,-5.0,172059.0,1.0,-5.0,100.0,NotFound,27.0,New,NotFound,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
134830,3577529,-20.0,632381.0,-1.0,-36.0,100.0,NotFound,27.0,New,NotFound,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
134831,3577531,-5.0,55528.0,0.0,-7.0,100.0,NotFound,27.0,Found,Found,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Define X and y
from sklearn.model_selection import train_test_split

X = df_join.drop(columns=['TransactionID', 'isFraud'], inplace=False) # drop id and label
y = df_join['isFraud']

# X.dtypes.unique()

In [19]:
# define a function to get dummies for the catogorical cols.
def get_dummies(df: DataFrame, cols: list):
  '''
  Get the dummy values for the categorical columns.
  Append them to the input df and drop the original cols.

  df: data.
  cols: the name of the columns need to be converted.
  '''

  for col in cols:
    if col in df.columns:
      col_dummies = pd.get_dummies(data=df[col])
      df = pd.concat([df, col_dummies], axis=1)
      df = df.drop(col, axis=1)

  
  return df

In [20]:
# for the str type cols in X, filter out them, 
# transform datatype to 'category'
# then get dummies for all these cols.
obj_cols = []
for col in X.columns:
  if X[col].dtype == 'O':
    X[col] = X[col].astype('category')
    obj_cols.append(col)
# print(obj_cols)

X = get_dummies(X, obj_cols)

X.shape

(134833, 2468)

In [21]:
X.columns

Index(['id_01', 'id_02', 'id_05', 'id_06', 'id_11', 'id_13', 'id_17', 'id_19',
       'id_20', 'TransactionDT',
       ...
       'NA', 'M4_M0', 'M4_M1', 'M4_M2', 'M4_NA', 'NA', 'NA', 'NA', 'NA', 'NA'],
      dtype='object', length=2468)

In [22]:
# rename the duplicates named cols.
cols = pd.Series(X.columns)
dup_count = cols.value_counts()
for dup in cols[cols.duplicated()].unique():
    cols[cols[cols == dup].index.values.tolist()] = [dup + str(i) for i in range(1, dup_count[dup]+1)]

# run it twice, because newly named cols in last step got dups with ori not changed col names.
X.columns = cols
cols = pd.Series(X.columns)
dup_count = cols.value_counts()
for dup in cols[cols.duplicated()].unique():
    cols[cols[cols == dup].index.values.tolist()] = [dup + str(i) for i in range(1, dup_count[dup]+1)]

X.columns = cols

In [23]:
# test for if there are still duplicates names in the df
uni_set = set()
for col in cols:
  if col not in uni_set:
    uni_set.add(col)
  else:
    print(col)

len(cols) - len(uni_set)

0

###5. Split to train, val and test datasets.


In [24]:
X_train, X_test, y_train, y_test\
    = train_test_split(X, y, test_size=0.2, random_state=697)

X_train, X_val, y_train, y_val\
    = train_test_split(X_train, y_train, test_size=0.25, random_state=697) # 0.25 x 0.8 = 0.2

In [25]:
X_train.shape
X_train.columns

Index(['id_01', 'id_02', 'id_05', 'id_06', 'id_11', 'id_13', 'id_17', 'id_19',
       'id_20', 'TransactionDT',
       ...
       'NA11', 'M4_M0', 'M4_M1', 'M4_M2', 'M4_NA', 'NA12', 'NA13', 'NA14',
       'NA15', 'NA16'],
      dtype='object', length=2468)

###6. Fill null values_B_with mean and most frequent values. fit and train using pipeline.

    1.b fill with **mean** value of the column
>numerical 'id_XX'

    2.b fill with **mean** value of the column using sklearn.impute.

>card1 - card6: payment card information.Such as card type, card category, issue bank, country, etc.

>addr: both addresses are for purchaser.
addr1 as billing region.
addr2 as billing country.

>dist: distances between (not limited) billing address, mailing address, zip code, IP address, phone area, etc.
<br>

    2.c fill with **most frequent value** in the column.
  >C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
  
  >D1-D15: timedelta, such as days between previous transaction, etc.
<br>

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Step 1:
# filter out the cols with Nans.
X_null = X.isnull().sum(axis=0).to_frame() # count Nans in every col.
X_null.rename(columns={0: '#_Nans'}, inplace=True) # rename cols.
X_NanCols = X_null[X_null['#_Nans']>0].index # get a series contains all the names of cols with Nan.

X_fullCols = X_null[X_null['#_Nans']==0].index

# make lists, indicating which stratage will be used in imputing the cols.
cols_fill_mean = []
cols_fill_freq = []

for col in X_NanCols:
  if str(col).startswith('C'): # cols C1-C1
    cols_fill_freq.append(col)
  elif str(col).startswith('D'): # cols D1-D15 and 'Device ...' which has been filled previously.
    cols_fill_freq.append(col)
  else:
    cols_fill_mean.append(col) # cols id_XX and cols has already been filled with other startages earlier.

# make all the cols still included in the following processing
cols_fill_freq.extend(X_fullCols.to_list())

In [27]:
# cols_fill_freq

In [28]:
# Step 2:
# instantiate the imputers, within a pipeline
# imputer imputes with the mean
imp_mean = Pipeline(steps=[('imputer', SimpleImputer(missing_values=np.nan, strategy='mean'))])

# imputer imputes with 'most_frequent'
imp_freq = Pipeline(steps=[('imputer',SimpleImputer(missing_values=np.nan, strategy='most_frequent'))])


# Step 3:
# put the features list and the transformers together by col transformer.
imp_preprocessor = ColumnTransformer(transformers=[('imp_mean', imp_mean, cols_fill_mean),\
                                                   ('imp_freq',imp_freq,cols_fill_freq)])

In [29]:
# Step 4:
# fit and trans the datasets with 'imp_preprocessor'.
imp_preprocessor.fit(X_train)

X_train = imp_preprocessor.transform(X_train)
X_val = imp_preprocessor.transform(X_val)
X_test = imp_preprocessor.transform(X_test)

In [30]:
X_val.shape

(26967, 2466)

In [31]:
X_train[1]


array([ 8.225e+03,  1.000e+00, -6.000e+00, ...,  1.000e+00,  1.000e+00,
        1.000e+00])

In [32]:
# output
X_train.tofile('X_train')
!cp X_train "drive/MyDrive/Capstone/Data/"

X_val.tofile('X_val')
!cp X_val "drive/MyDrive/Capstone/Data/"

X_test.tofile('X_test')
!cp X_test "drive/MyDrive/Capstone/Data/"




In [34]:
y_train.to_csv('y_train.csv')
!cp y_train.csv "drive/MyDrive/Capstone/Data/"

y_val.to_csv('y_val.csv')
!cp y_val.csv "drive/MyDrive/Capstone/Data/"

y_test.to_csv('y_test.csv')
!cp y_test.csv "drive/MyDrive/Capstone/Data/"